In [1]:
import requests
import pandas as pd
from pathlib import Path
from os import environ
import tqdm.contrib.concurrent as tqdm
import pickle

In [2]:
root = Path(':'.join(environ.get("PYTHONPATH", ".").split(":")[:2]))
data = root / 'data' / 'MO'

In [57]:
df = pd.read_csv(data / 'MO_images.csv')
train = pd.read_csv('../training/data/train.csv')

In [ ]:
list403 = []

def download(url):
    response = requests.get(url)
    if response.status_code == 200:
        with open(data / 'images' / Path(url).name, 'wb') as handle:
            handle.write(response.content)
        pass
    else:
        list403.append(Path(url).name)

In [ ]:
(data / 'images').mkdir(exist_ok=True)
tqdm.thread_map(download, df['image'].values.tolist(), max_workers=500)

pickle.dump(list403, open(data / 'list4032.pkl', 'wb'))
df = df.drop(df[df['file_name'].isin(list403)].index)

In [ ]:
df['file_name']= df['image'].str.split('/').str[-1]
df['file_path'] = str(data / 'images') + '/' + df['file_name']

df.loc[:, ["family", "genus", "class", "order", "phylum"]] = df.loc[:, ["family", "genus", "class", "order", "phylum"]].replace("[',]", "", regex=True)
df = df[df['name'].str.split(' ').str.len() > 1]
df.loc[df['dataset'] == '2019', "file_path"] = df.loc[df['dataset'] == '2019', "file_path"].replace(r"train_val2019/", '', regex=True)

df = df.drop_duplicates(subset='file_name')
df = df.replace(f'^(https?://)([^/]+)/images(/.+)', r'\1images.\2\3', regex=True)

df[['height', 'width']] = 480, 640
df['dataset'] = 'MO'

df.drop(['created', 'license', 'rightsHolder'], axis=1, inplace=True)

full = pd.concat([train.drop(['specific_epithet', 'dset', 'class_id'], axis=1), df.drop('image', axis=1)], ignore_index=True)
full['genus'] = full['name'].str.split(' ').str[0]

In [44]:
def fill(df, column, fam, order, clas, phylum):
    df.loc[df['genus'] == column, 'family'] = fam
    df.loc[df['genus'] == column, 'order'] = order
    df.loc[df['genus'] == column, 'class'] = clas
    df.loc[df['genus'] == column, 'phylum'] = phylum
    df.loc[df['genus'] == column, 'kingdom'] = "Fungi"
    return df

In [43]:
full = full.fillna(full.groupby("genus").transform(lambda x: x.mode().iloc[0]))

with open('fill_taxa.txt') as f:
    for line in f:
        line = line.split()
        line = [x.replace('",', '').replace('"', '').replace(')', '') for x in line]
        full = fill(full, line[3], line[4], line[5], line[6], line[7])

kingdom      57888
name             0
family       58225
phylum       57888
order        58019
genus            0
class        57888
file_name        0
height           0
width            0
dataset          0
file_path        0
dtype: int64

In [ ]:
!ls ../training/data/images > ../training/data/file_list.txt

with open(data / 'file_list.txt', 'r') as f:
    file_list = f.read().splitlines()

mo = full.loc[full['dataset'] == 'MO']
other = full.loc[full['dataset'] != 'MO']

file_list = set(file_list)
fset = set(mo['file_name'])
missing = fset - file_list

full = full.loc[~full['file_name'].isin(missing)]

In [ ]:
empty_list = []
for file in file_list:
    if (data / 'images' / file).stat().st_size < 1000:
        empty_list.append(line)

full = full.loc[~full['file_name'].isin(empty_list)]

In [46]:
full['class_id'] = full['name'].astype('category').cat.codes
class_dict = full.set_index('class_id')['name'].to_dict()

full.to_csv(root / 'training' / 'data' / 'train_full.csv', index=False)
pickle.dump(class_dict, open(root / 'training' / 'data' / 'class_dict_full.pkl', 'wb'))

In [30]:
group = full.groupby("name").size().sort_values()
names = group[group > 50].reset_index()['name'].to_list()
full = full[full['name'].isin(names)]

full['class_id'] = full['name'].astype("category").cat.codes

full.to_csv(root / 'training' / 'data' / 'train_full_smaller.csv', index=False)
pickle.dump(full.set_index('class_id')['name'].to_dict(), open(root / 'training' / 'data' / 'class_dict_full_smaller.pkl', 'wb'))